In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
import pandas as pd
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

from util import remove_fullspace_and_newline, reweight_distribution

C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [3]:
# 歌詞データの読み込み
df = pd.read_csv('./datasets/all_songs.csv')
lyrics = df['Lyric']
lyrics.head()

0    まるで荒れる波涛のように背筋つらぬき　心狂わす出逢いそう　出逢い夢うつつと見紛(みまご)うほ...
1    Don't Don't Don't Stay Good-byeDon't Don't Don...
2    YES♪広い空のようなみんな夢を見てるそして叶えてく輝くこの宇宙(そら)でどこ行(ゆ)こうM...
3    いっしょに歌おういっしょの時代(とき)の中いっしょで行こういっしょな夢を見よういっしょにいよ...
4    アッ！とね　言わせて見たいいっぱい愛があふれるウットリするような世界創ろうガッカリするのまだ...
Name: Lyric, dtype: object

In [4]:
lyrics[0]

"まるで荒れる波涛のように背筋つらぬき\u3000心狂わす出逢いそう\u3000出逢い夢うつつと見紛(みまご)うほどあどけなさに吸い込まれるような瞳そう\u3000瞳(そうよキミ\u3000近づいてきて至近距離\u3000手が届くまで)パラダイムが一新されてくの胸にたぎる黒い鼓動は目の前に立つ必死の顔が由来そう\u3000由来どんなふうに捕まえたい？と甘やかすぎる戸惑い\u3000そして喜劇そう\u3000喜劇(いいわキミ\u3000そうよまっすぐなけなしの\u3000気合を乗せて)紅白こそ罪だと教えるわ今\u3000この恋愛感情の\u3000昂(たかぶ)るままに命じるの強く\u3000嗚呼！(I want！)そこに跪(ひざまず)いて！(I need！)認めたいの\u3000あなたを(I make！I love！)私のやり方で月も恥じるこの舞台で\u3000(woo woo I've got desire)忘れられぬ夢よ今夜\u3000(woo woo I've got desire)鞭打つわ真っ赤な証明\u3000(woo woo I've got desire)もっともっと燃え上がれ怖い夢\u3000イタイイタイアマイ(言ってキミ\u3000誓いの言葉ありったけ\u3000本気の声で)後悔などさせないわ\u3000ここから今\u3000その全身全霊に無窮の愛と\u3000強い支配を刻むのよ世界をすべて振り捨ててでも究(きわ)めあげたい\u3000はじめての他の何にも替わることない最高の瞬間(とき)よ\u3000嗚呼！(I want！)もっと啼(な)いてみせて！(I need！)離さないわ\u3000二度とは(I make！You love！)至福の地獄からどこまで\u3000堕ちる\u3000堕ちる\u3000このままふたりで\u3000いける\u3000いける\u3000高みへどれだけ\u3000燃える\u3000燃える\u3000一途に求めて\u3000翔べる\u3000翔べる\u3000どこまで！"

In [5]:
removed = remove_fullspace_and_newline(lyrics[0])
removed

"まるで荒れる波涛のように背筋つらぬき心狂わす出逢いそう出逢い夢うつつと見紛(みまご)うほどあどけなさに吸い込まれるような瞳そう瞳(そうよキミ近づいてきて至近距離手が届くまで)パラダイムが一新されてくの胸にたぎる黒い鼓動は目の前に立つ必死の顔が由来そう由来どんなふうに捕まえたい？と甘やかすぎる戸惑いそして喜劇そう喜劇(いいわキミそうよまっすぐなけなしの気合を乗せて)紅白こそ罪だと教えるわ今この恋愛感情の昂(たかぶ)るままに命じるの強く嗚呼！(I want！)そこに跪(ひざまず)いて！(I need！)認めたいのあなたを(I make！I love！)私のやり方で月も恥じるこの舞台で(woo woo I've got desire)忘れられぬ夢よ今夜(woo woo I've got desire)鞭打つわ真っ赤な証明(woo woo I've got desire)もっともっと燃え上がれ怖い夢イタイイタイアマイ(言ってキミ誓いの言葉ありったけ本気の声で)後悔などさせないわここから今その全身全霊に無窮の愛と強い支配を刻むのよ世界をすべて振り捨ててでも究(きわ)めあげたいはじめての他の何にも替わることない最高の瞬間(とき)よ嗚呼！(I want！)もっと啼(な)いてみせて！(I need！)離さないわ二度とは(I make！You love！)至福の地獄からどこまで堕ちる堕ちるこのままふたりでいけるいける高みへどれだけ燃える燃える一途に求めて翔べる翔べるどこまで！"